In [5]:
import numpy as np

In [6]:
torchat_vectors = np.load("torchat_context_vectors.npy")

Применим K-Means кластеризацию

сначала 3 значения (1 уровень), потом 9 значений (2 уровень), потом 11 значений (см. Презентацию)

In [1]:
from sklearn.cluster import KMeans

In [3]:
clusterizer = KMeans(n_clusters=3, random_state=42)

In [7]:
cluster_labels = clusterizer.fit_predict(torchat_vectors)

Посмотрим на сумму внутригрупповых дисперсий:

In [8]:
clusterizer.inertia_

266362.87362255756

Алгоритм сошёлся всего за 19 шагов:

In [9]:
clusterizer.n_iter_

19

In [10]:
import json

In [12]:
with open('torchat.json', 'r', encoding='utf-8') as inp:
    torchat_contexts = json.load(inp)

In [15]:
clustered_torchat_contexts = [[context for index,
                               context in enumerate(torchat_contexts) if cluster_labels[index]==cluster]for cluster in set(cluster_labels)]

In [14]:
set(cluster_labels)

{0, 1, 2}

Посмотрим, какого объёма получились кластеры:

In [16]:
for cluster in clustered_torchat_contexts:
    print(len(cluster))

917
809
838


Посмотрим на центры кластеров:

In [19]:
clusterizer.cluster_centers_.shape

(3, 300)

In [22]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [23]:
from collections import defaultdict

In [41]:
def get_vectors_and_contexts(verb):
    vectors = np.load(verb+"_context_vectors.npy")
    
    with open(verb+'.json', 'r', encoding='utf-8') as inp:
        contexts = json.load(inp)
    
    return vectors, contexts

Будем выбирать наиболее характерные для кластера слова при помощи Хи-квадрат теста:

In [31]:
from scipy.stats import chi2_contingency as chi2

In [32]:
def get_clustered_contexts(vectors, contexts, clusterizer):
    cluster_labels = clusterizer.fit_predict(vectors)
    clustered_contexts = [[context for index,
                           context in enumerate(contexts) if cluster_labels[index]==cluster]for cluster in set(cluster_labels)]
    freqs = defaultdict(int)
    sum_words = 0
    sum_list = []
    freq_lists = []
    
    for cluster in clustered_contexts:
        local_sum = 0
        freqlist = defaultdict(int)
        for context in cluster:
            for word in context[0]:
                freqlist[word] += 1
                freqs[word] += 1
                sum_words += 1
                local_sum += 1
        freq_lists.append(freqlist)
        sum_list.append(local_sum)
    
    keyword_lists = []
    for index, freqlist in enumerate(freq_lists):
        keyword_list = dict()
        for word in freqlist:
            keyword_list[word] = chi2([[freqlist[word], freqs[word] - freqlist[word]],
                                       [sum_list[index] - freqlist[word], sum_words - freqs[word]]])[0]
        keyword_list = [(key, keyword_list[key]) for key in sorted(keyword_list, key = lambda x: -keyword_list[x])]
        keyword_lists.append(keyword_list)
    return clustered_contexts, keyword_lists

In [68]:
torchat_contexts_3, torchat_keywords3 = get_clustered_contexts(torchat_vectors, torchat_contexts,
                                                            KMeans(n_clusters=3, random_state=42))

In [69]:
for clist in torchat_contexts_3:
    print(len(clist))

917
809
838


In [36]:
torchat_keywords3[0][:20]

[(',_PUNCT', 288.6831305164844),
 ('рука_NOUN', 236.91852730650245),
 ('волос_NOUN', 206.96061568633894),
 ('нога_NOUN', 170.79968105702892),
 ('._PUNCT', 163.02102348680046),
 ('голова_NOUN', 161.2272443385655),
 ('глаз_NOUN', 127.6278417981747),
 ('из_ADP', 121.47083300421043),
 ('из-под_ADP', 111.31098331157926),
 ('нос_NOUN', 108.68512894946782),
 ('с_ADP', 94.77953394803394),
 ('его_DET', 85.52452116871223),
 ('лицо_NOUN', 83.54419522328955),
 ('ухо_NOUN', 78.36920236418487),
 ('и_CCONJ', 74.69710551517637),
 ('он_PRON', 73.27454118519447),
 ('палец_NOUN', 69.88726178737234),
 ('рот_NOUN', 69.55859898318774),
 ('на_ADP', 65.72407242983455),
 ('длинный_ADJ', 64.72406976405495)]

In [37]:
torchat_keywords3[1][:20]

[(',_PUNCT', 174.65577878003964),
 ('земля_NOUN', 133.65500273294904),
 ('вода_NOUN', 126.76106727609908),
 ('на_ADP', 110.23947459996016),
 ('из_ADP', 103.55177472855586),
 ('дерево_NOUN', 91.49809832142117),
 ('._PUNCT', 83.87184898945466),
 ('берег_NOUN', 76.25856593631639),
 ('труба_NOUN', 72.91703274934159),
 ('стена_NOUN', 72.10378317155075),
 ('крыша_NOUN', 71.16242785327748),
 ('железный_ADJ', 53.85722256368671),
 ('и_CCONJ', 53.190379335017404),
 ('снег_NOUN', 50.481808373189295),
 ('ржавый_ADJ', 48.897582781376975),
 ('метр_NOUN', 45.404216093431984),
 (';_PUNCT', 43.223801375277716),
 ('над_ADP', 42.29446373959268),
 ('лист_NOUN', 38.38636969077886),
 ('волна_NOUN', 38.38636969077886)]

In [40]:
torchat_keywords3[2][:20]

[(',_PUNCT', 285.48591181202687),
 ('не_PART', 241.9968164181772),
 ('я_PRON', 216.56690285948525),
 ('в_ADP', 155.00009443024692),
 ('бы_PART', 100.67423082473745),
 ('?_PUNCT', 99.72825566414178),
 ('―_PUNCT', 90.55808568533295),
 ('что_PRON', 89.6782487151146),
 ('что_SCONJ', 83.67930650230623),
 ('мы_PRON', 74.05164508649585),
 ('все_PRON', 67.86096990326556),
 ('ты_PRON', 64.56374771904234),
 (':_PUNCT', 64.44198794980304),
 ('день_NOUN', 64.14825012078326),
 ('жизнь_NOUN', 62.0886904347662),
 (')_PUNCT', 59.874712894659254),
 ('время_NOUN', 59.8461865094594),
 ('и_CCONJ', 55.939659140191566),
 ('так_ADV', 55.122072282980604),
 ('же_PART', 53.12594686277202)]

Попробуем то же самое, но не с K-Means, а с AgglomerativeClustering:

In [42]:
from sklearn.cluster import AgglomerativeClustering

In [56]:
torchat_contexts3_aggl, torchat_keywords3_aggl = get_clustered_contexts(torchat_vectors, torchat_contexts,
                                                             AgglomerativeClustering(n_clusters=3))

In [57]:
torchat_keywords3_aggl[0][:20]

[(',_PUNCT', 569.2461239977644),
 ('на_ADP', 189.10210821813632),
 ('и_CCONJ', 153.95047273711447),
 ('._PUNCT', 132.76197935426984),
 ('из_ADP', 123.40668378741955),
 ('вода_NOUN', 120.80898072516923),
 ('в_ADP', 90.90534054347854),
 ('по_ADP', 80.80489252618057),
 (';_PUNCT', 75.89954801734122),
 ('с_ADP', 65.30372012022404),
 ('который_PRON', 59.64032186204452),
 ('берег_NOUN', 58.90097288573857),
 ('труба_NOUN', 56.65814511463664),
 (')_PUNCT', 54.087550825374194),
 ('как_SCONJ', 51.26803858040496),
 ('стена_NOUN', 47.23422757533159),
 ('земля_NOUN', 45.26261530510217),
 ('крыша_NOUN', 44.4935946100202),
 ('дерево_NOUN', 44.21873375463393),
 ('―_PUNCT', 42.51182570352203)]

In [58]:
torchat_keywords3_aggl[1][:20]

[('не_PART', 166.14222850881413),
 ('я_PRON', 138.16189284725223),
 ('что_PRON', 84.71888298926355),
 ('?_PUNCT', 84.01325668144229),
 ('мы_PRON', 80.64818194190454),
 ('ты_PRON', 72.34639582975741),
 ('день_NOUN', 66.20984768710869),
 ('жизнь_NOUN', 62.74230145070284),
 ('―_PUNCT', 59.09890152194406),
 ('вечер_NOUN', 54.549745413974705),
 ('утро_NOUN', 54.092750947767975),
 ('что_SCONJ', 51.60225121589441),
 ('из_ADP', 51.4733582762069),
 (',_PUNCT', 48.982284793657804),
 ('звонить_VERB', 48.78166679689383),
 ('в_ADP', 47.130038587448475),
 ('знать_VERB', 44.212852464368034),
 ('понимать_VERB', 43.71125595458259),
 ('здесь_ADV', 43.37024173583975),
 ('бы_PART', 40.51621971503351)]

In [59]:
torchat_keywords3_aggl[2][:20]

[(',_PUNCT', 322.01408311759405),
 ('._PUNCT', 229.73328267020165),
 ('волос_NOUN', 183.5683883924818),
 ('из_ADP', 176.43343114320405),
 ('голова_NOUN', 173.60499843747417),
 ('рука_NOUN', 170.12020200591266),
 ('глаз_NOUN', 143.92197128967211),
 ('нога_NOUN', 126.36209673104267),
 ('с_ADP', 114.78560320042375),
 ('из-под_ADP', 100.03371115256093),
 ('нос_NOUN', 90.4294495268168),
 ('в_ADP', 84.25303871058723),
 ('и_CCONJ', 79.62414614501819),
 ('лицо_NOUN', 75.79796607244431),
 ('ухо_NOUN', 74.60752962567527),
 ('сторона_NOUN', 74.59788805565033),
 ('на_ADP', 66.27326113580561),
 ('он_PRON', 64.7037634045296),
 ('белый_ADJ', 61.72963682103258),
 ('грудь_NOUN', 58.56654531857992)]

Не, дичь какая-то получается, продолжить работать с K-Means

In [90]:
def visualize(contexts, keywords, n):
    for idx, keywordlist in enumerate(keywords):
        print(idx, ', '.join([i[0].split('_')[0] for i in keywordlist[:n]]), len(contexts[idx]))

In [62]:
torchat_contexts9, torchat_keywords9 = get_clustered_contexts(torchat_vectors, torchat_contexts,
                                                             KMeans(n_clusters=9, random_state=42))

In [91]:
visualize(torchat_contexts9, torchat_keywords9, 20)

0 волос, глаз, усы, лицо, седой, борода, рубашка, белый, ухо, нос, синий, бородка, затылок, черный, рыжий, длинный, шапка, шея, воротник, подбородок 264
1 ,, так, она, он, и, я, не, :, ты, слышать, что, тетя, ―, …, кричать, стать, рассказывать, смотреть, кто, держать 467
2 плод, ), представлять, средний, длина, лопасть, клетка, более, безопасность, соединить, окраска, молекула, вид, поверхность, ;, искусственный, растение, членик, апикальный, конусовидный 307
3 крыша, метр, вышка, стена, железный, башня, труба, кирпичный, колесо, ржавый, балка, яма, рельс, пулемет, решетка, кабель, река, слева, штырь, бетонный 182
4 вода, дерево, земля, снег, берег, трава, море, озеро, камень, гора, дождь, корень, ветер, пальма, островок, деревце, ель, среди, волна, куст 288
5 нога, рука, палец, голова, грудь, рот, ладонь, комок, ., из, кол, нож, живот, плечо, рана, колено, нос, рукоятка, кисть, лопатка 275
6 день, вечер, час, утро, ночь, дома, ?, пост, целый, звонить, сутки, домой, водитель, и, торчит

In [84]:
def easy_visualize(verb, n, k):
    visualize(*get_clustered_contexts(*get_vectors_and_contexts(verb),
              KMeans(n_clusters=n, random_state=42)),
              k)

In [81]:
visualize(*get_clustered_contexts(torchat_vectors, torchat_contexts,
                                 KMeans(n_clusters=11, random_state=42)), 12)

0 [('рубашка_NOUN', 169.71269272832336), ('из-под_ADP', 114.3346156915463), ('куртка_NOUN', 106.33122540964305), ('рукав_NOUN', 90.31661892004401), ('шапка_NOUN', 83.99886421105215), ('пальто_NOUN', 81.88929612900655), ('костюм_NOUN', 81.17856584116078), ('пиджак_NOUN', 75.75690335174481), ('штаны_NOUN', 67.52533542779764), ('кепка_NOUN', 66.5511443794471), ('вязаный_ADJ', 60.20107708277208), ('козырек_NOUN', 60.20107708277208)] 162
1 [('нога_NOUN', 240.58317474813325), ('рука_NOUN', 209.88615361875586), ('палец_NOUN', 108.58522048550809), ('грудь_NOUN', 88.4789470622657), ('ладонь_NOUN', 82.28944882139103), ('плечо_NOUN', 68.88853640967905), ('рукоятка_NOUN', 67.87218751653246), ('голова_NOUN', 65.94868110896293), ('нож_NOUN', 59.948636811554636), ('._PUNCT', 58.71094850932727), ('колено_NOUN', 55.12728224499956), ('рот_NOUN', 49.93566480920614)] 173
2 [('я_PRON', 90.23915446284693), ('не_PART', 75.86022857890131), ('знать_VERB', 53.4182065156635), ('?_PUNCT', 52.67411971764224), ('бы

In [92]:
easy_visualize('vystupat', 9, 12)

0 чемпионат, команда, сборная, год, спорт, мастер, украинец, беспартийный, рядовой, стрелковый, 1941, игрок 151
1 государство, партия, политика, политический, против, власть, правительство, Гоминдан, суд, защита, независимость, держава 185
2 компания, развитие, проект, банк, региональный, система, рынок, банковский, медицинский, продажа, международный, сотрудничество 205
3 концерт, театр, оркестр, музыкант, зал, костюм, песня, оперный, артист, знаменитый, ансамбль, солист 143
4 глаз, ,, ―, он, рука, улыбаться, слёз, тут, я, точно, губа, слеза 177
5 вода, стена, поверхность, из, над, черный, край, пластинка, камень, узкий, лезвие, угол 109
6 научный, доклад, конференция, заседание, делегат, сотрудник, журнал, заместитель, совет, кафедра, статья, В. 184
7 качество, форма, исходный, язык, процесс, более, элемент, понятие, символ, организм, смысл, событие 180
8 ,, не, что, я, жизнь, говорить, ?, знать, много, быть, но, о 313


In [93]:
easy_visualize('igrat', 11, 12)

0 команда, клуб, сборная, год, чемпионат, матч, ", в, турнир, бесплатно, трижды, убыток 197
1 роль, рынок, важный, ключевой, страна, экономика, организация, США, вирус, газ, банк, экономический 215
2 фильм, пьеса, театр, спектакль, актер, артист, художник, комедия, кино, персонаж, режиссер, театральный 162
3 солнце, берег, ветер, небо, вода, дерево, золотой, освещать, свирель, камень, стена, свет 145
4 не, слово, это, я, но, думать, уметь, знать, потому, любить, бы, игра 254
5 ?, сказать, вы, она, он, я, жена, хотеть, ,, ты, любить, сын 322
6 гитара, музыка, вальс, оркестр, рояль, танец, песня, петь, звучать, марш, джаз, музыкант 114
7 ,, 4, поле, без, ", ), называть, компьютер, соперник, выглядеть, не, 13 362
8 роль, важный, понятие, воспитание, церковь, мир, жизнь, религиозный, духовный, условие, общество, политический 246
9 двор, вечер, бегать, дом, сад, час, пить, с, ходить, этаж, чай, роль 237
10 глаз, нога, сидеть, зуб, рука, дверь, палец, лошадь, сено, косичка, губа, внизу 198
